<a href="https://colab.research.google.com/github/Mohamed28112003/Muffakir/blob/main/DATA_GENERATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers
!pip install -U bitsandbytes
!pip install langchain_groq
!pip install langchain
!pip install -U langchain-community

Your final data frame

In [ ]:
import pandas as pd
final_df = pd.DataFrame()


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust chunk size based on content length
    chunk_overlap=0,  # No overlap needed
    separators=["Page number"] # Split at answer or next question
)




from langchain.document_loaders import TextLoader

# Load documents from the specified directory
data_loader = TextLoader('/content/drive/MyDrive/Data/حقوق /عين شمس /رابعه/TXT/اجراءات_الاستدلال_والتحقيق_أ_د_جميل_عبد_الباقي_الصغير.txt')
data = data_loader.load_and_split(text_splitter=text_splitter_recursive) # Load documents
print(f"Documents : {len(data)}.")


Documents : 395.


In [ ]:
print(data[185].page_content)

In [ ]:
selected_data = data[1:390]
## DO NOT FORGET TO EDIT THE NUMBERS HERE AND ALSO NEGLECT ANY USEFUL PAGES

In [ ]:
from langchain_groq import ChatGroq


In [ ]:

# Initialize the ChatGroq model
llm = ChatGroq(
    api_key="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",  # Retrieve the API key from the environment variable
    model="llama-3.1-70b-versatile",
    temperature=0.7,
    max_tokens=1024,
)

In [ ]:
#  you can optimize this prompt if you need but keep it with the smae format please :)

prompt = """

Based on the following text, generate one Question and its corresponding Answer.
For each question you formulate, provide a comprehensive answer with details immediately following the question, separated by a vertical bar "|" provide only 2-3 questions. The context to analyze is as follows: <Context> {context} </Context>.
For Example :
## Context:
البعيد أو الغائي لقانون الإجراءات الجنائية يمثل ذات الهدف بالنسبة لكافة العلوم الجنائية.
رابعاً - وظيفة قانون الإجراءات الجنائية:
يمكن القول بأن الوظيفة الأساسية لهذا القانون هي تنظيم العدالة الجنائية؛ وتحقيق هذه
الغاية يتطلب الموائمة بين اعتبارين. يتمثل الاعتبار الأول في وضع آلية يُمكن من خلالها
تفعيل نصوص قانون العقوبات حماية للمصالح الاجتماعية العامة المتعددة، ويتجلى الاعتبار
الثاني في ضمان حقوق الفرد وحريته في مواجهة السلطات الإجرائية الجنائية.

# Your Response Sholud Be Like This format only do not add any thing else
Question : وظيفة قانون الإجراءات الجنائية | يمكن القول بأن الوظيفة الأساسية لهذا القانون هي تنظيم العدالة الجنائية؛ وتحقيق هذه
الغاية يتطلب الموائمة بين اعتبارين. يتمثل الاعتبار الأول في وضع آلية يُمكن من خلالها
تفعيل نصوص قانون العقوبات حماية للمصالح الاجتماعية العامة المتعددة، ويتجلى الاعتبار
الثاني في ضمان حقوق الفرد وحريته في مواجهة السلطات الإجرائية الجنائية.



"""

In [ ]:
from langchain.schema import HumanMessage  # Import HumanMessage


In [ ]:
import pandas as pd

# List to hold the structured data
results = []
i=0
for doc in selected_data:
    Final_prompt = prompt.format(context=doc.page_content)
    messages = [HumanMessage(content=Final_prompt)]
    response = llm.invoke(messages)

    # Split the response into individual question-answer pairs
    qa_pairs = response.content.split("Question :")
    print("Counter:", i)
    # IF YOU WANT TO SEE THE PAGE CONTENT UN COMMENT THIS LINE
    #print("Context  :  ",doc.page_content)

    i+=1
    for qa in qa_pairs[1:]:  # Skip the first empty part

        question_answer = qa.split(" | ")
        if len(question_answer) == 2:
            question = question_answer[0].strip()
            answer = question_answer[1].strip()
            print("Question:", question)
            print("Answer:", answer)
            # print("-------------------------------------------------------------------")
            # print("page_content")
            # print(doc.page_content)

            results.append({
                "context": doc.page_content,
                "question": question,
                "answer": answer
            })



In [ ]:

# Convert results to a DataFrame
df = pd.DataFrame(results)


In [ ]:
df.tail()

In [ ]:
df.shape

(340, 3)

In [ ]:


print("Context:\n", df.loc[2, "context"])

print("Question:\n", df.loc[2, "question"])
print("Answer:\n", df.loc[2, "answer"])


In [ ]:

pd.set_option('display.max_colwidth', None)


In [ ]:
df.tail(10)


In [ ]:
# CONCAT THE DF WITH TH FINAL_DF

final_df = pd.concat([final_df, df], ignore_index=True)



# Run this cell after you finish all the pages of the book

In [ ]:
# replace the book name
from google.colab import files


final_df.to_csv('Book Name.csv', index=False)

files.download('Book Name.csv')